In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime
import time
import urllib
import json

app_id = "443809049300463"
app_secret = "e6ff2a431bb3da7624faefbf39a15a3d" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret
access_token

test_post_url = pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][0]

In [3]:
def request_until_succeed(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

#this one differs in the format:
# the output data should be post, user_id, user_name, and type
def parseReactions(post_id, num_reactions):
    base = "https://graph.facebook.com/v2.8/"+post_id+"/reactions"
    parameters = "?access_token=%s&limit=%s" % (access_token, num_reactions)
    
    url = base + parameters
    
    
    scrape_starttime = datetime.datetime.now()
    data = json.loads(request_until_succeed(url).decode('utf-8'))
    
    full_data = []
    pages = 0
    has_next_page = True
    
    
    while has_next_page:
        full_data = full_data + data['data']
        pages+=1
        # if there is no next page, we're done.
        if 'next' in data['paging'].keys():
            data = json.loads(request_until_succeed(data['paging']['next']).decode('utf-8'))
        else:
            has_next_page = False
            
    print("\nDone!\n%s pages Processed in %s" % (pages, datetime.datetime.now() - scrape_starttime))
            
    return {'_id' : post_id, 'reactions' : full_data}

In [8]:
client = MongoClient()
db = client.meme_scrape
# for i in range(2):
#     x = parseAllReactions(pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][i], 100)
#     end = db.reactions_full.insert_one(x)
db.user_reactions.drop()

In [93]:
def parseAllReactions(post_id, num_reactions):
    base = "https://graph.facebook.com/v2.8/"+post_id+"/reactions"
    parameters = "?access_token=%s&limit=%s" % (access_token, num_reactions)
    
    url = base + parameters
    
    scrape_starttime = datetime.datetime.now()
    data = json.loads(request_until_succeed(url).decode('utf-8'))
    
    user_data = []
    pages = 0
    has_next_page = True
    
    while has_next_page:
        for i in data['data']:
            db.user_reactions.update_one( { "_id": i['id'] } , 
                                        { "$push": { 'reactions' : {'type' : user['type'], 'post_id' : test_post_url} } }, upsert = True)
        pages+=1
        # if there is no next page, we're done.
        if 'next' in data['paging'].keys():
            data = json.loads(request_until_succeed(data['paging']['next']).decode('utf-8'))
        else:
            has_next_page = False
            
    print("\nDone!\n%s pages Processed in %s" % (pages, datetime.datetime.now() - scrape_starttime))

In [97]:
for i in range(30):
    parseAllReactions(pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][i], 100)


Done!
55 pages Processed in 0:00:07.765338

Done!
18 pages Processed in 0:00:02.822740

Done!
11 pages Processed in 0:00:02.051808

Done!
31 pages Processed in 0:00:05.973209
<urlopen error [Errno -2] Name or service not known>
Error for URL https://graph.facebook.com/v2.8/1717731545171536_1900035803607775/reactions?access_token=443809049300463%7Ce6ff2a431bb3da7624faefbf39a15a3d&limit=100&after=TVRBd01EQXdPRFF3TnpVMk1UZAzVPakUwT1RJMk56STJPREU2TWpVME1EazJNVFl4TXc9PQZDZD: 2017-04-20 19:41:55.274827

Done!
9 pages Processed in 0:00:21.804349

Done!
28 pages Processed in 0:00:09.668530

Done!
9 pages Processed in 0:00:02.350391

Done!
25 pages Processed in 0:00:05.766671

Done!
12 pages Processed in 0:00:02.849428

Done!
55 pages Processed in 0:00:20.821790

Done!
20 pages Processed in 0:00:03.274616

Done!
9 pages Processed in 0:00:06.657655

Done!
7 pages Processed in 0:00:01.748419

Done!
83 pages Processed in 0:00:26.663529

Done!
12 pages Processed in 0:00:02.557071

Done!
31 pages P

In [105]:
x = db.user_reactions.find({"_id" : '1881346522107516'})
for i in x:
    print(i)

{'_id': '1881346522107516', 'reactions': [{'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}, {'type': 'LIKE', 'post_id': '1717731545171536_1899560873655268'}]}


In [96]:
# clear everything cell
#db.user_reactions.drop()

In [6]:
a = parseReactions(test_post_url, 100)


Done!
55 pages Processed in 0:00:07.355195


In [13]:
db.test_reactions.drop()
db.test_reactions.insert_one(a)

In [14]:
s = db.test_reactions.find()
for i in s:
    print(i)

{'reactions': [{'type': 'LIKE', 'id': '297903657310531', 'name': 'Eduardo Corea-Dilbert'}, {'type': 'LIKE', 'id': '10155776456110476', 'name': 'Morgan Dunstan'}, {'type': 'LIKE', 'id': '10208221124956052', 'name': 'Chesna Pokharel'}, {'type': 'LIKE', 'id': '1064792573664262', 'name': 'Alexandria Storm'}, {'type': 'LIKE', 'id': '1470198023041109', 'name': 'Meghana Krishnakumar'}, {'type': 'LIKE', 'id': '10202867027712182', 'name': 'Zoe Yoo'}, {'type': 'LIKE', 'id': '824197767729870', 'name': 'Julia Culp'}, {'type': 'LIKE', 'id': '1462357797130965', 'name': 'Walker Hathaway-Williams'}, {'type': 'LIKE', 'id': '1721192314845010', 'name': 'Tim Dawson'}, {'type': 'LIKE', 'id': '1387646754607076', 'name': 'Amy Huang'}, {'type': 'LIKE', 'id': '10211415292449810', 'name': 'Katie Poa'}, {'type': 'LIKE', 'id': '276826202758804', 'name': 'Rasd Fder'}, {'type': 'LIKE', 'id': '10155025443876278', 'name': 'Caleb Luna'}, {'type': 'LIKE', 'id': '794240920739645', 'name': 'Deepthi Akella'}, {'type': 'LI